## Data Importing and Pre-processing

In [ ]:
# import libraries needed
import pandas as pd

pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm, skew, probplot
from scipy.special import boxcox1p
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.*")
%matplotlib inline

In [ ]:
# read file and see number of rows and cols
nba_df = pd.read_csv("nba_2022-23_all_stats_with_salary.csv")
nba_df.shape

In [ ]:
nba_df.head()

In [ ]:
# reanme 'Unnamed: 0' column to 'ID'
nba_df = nba_df.rename(columns={"Unnamed: 0": "Id"})

In [ ]:
# Remove spaces from column names
nba_df.columns = [col.replace(" ", "") for col in nba_df.columns]

In [ ]:
# count number of categorical variables
category_count = 0

for cat in nba_df.dtypes:
    if cat == "object":
        category_count += 1

In [ ]:
print("Number of categorical variables:", category_count)

# column 1 is the ID column so we subract 1
numeric_count = nba_df.shape[1] - category_count - 1

print("Number of contineous variables:", numeric_count)

In [ ]:
# see all the column names
nba_df.columns

### Handling our missing data

In [ ]:
# display the missing data and its percent of the column
total_missing = nba_df.isnull().sum().sort_values(ascending=False)
percent_missing = (nba_df.isnull().sum() / nba_df.isnull().count()).sort_values(ascending=False)

missing_data_df = pd.concat([total_missing, percent_missing], axis=1, keys=["Total Missing", "Percent Missing"])
missing_data_df.head(8)

In [ ]:
# example row of a player who has missing data
# players with missing data are those who did not play many games so they never accumilated that stat during the season
null_fg = nba_df[nba_df['FG%'].isnull()]
null_fg

In [ ]:
# visualize this in a bar graph
missing_data_df["Percent Missing"].head(8).plot(
    kind="barh", figsize=(20,10)
).invert_yaxis()
plt.xlabel("Percent Missing")
plt.ylabel("Variable")
plt.title("The 8 Columns and their Percent of Missing Data")
plt.show()

In [ ]:
# fill in the missing data with 0s
# data is "missing" because player never recorded that stat during the season so we impute that data to be 0 to identify them in our model
cols_to_fill_zero = [
    "FT%",
    "3P%",
    "2P%",
    "TS%",
    "3PAr",
    "FTr",
    "eFG%",
    "FG%",
]

for col in cols_to_fill_zero:
    nba_df[col] = nba_df[col].fillna(0)


In [ ]:
# show same player who had null values now has zeros in those fields
imputed_row = nba_df[nba_df["PlayerName"] == "Alondes Williams"]
imputed_row

### Handling outliers for better training

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x=nba_df["GP"], y=nba_df["Salary"])
plt.ylabel("Salary", fontsize=13)
plt.xlabel("GP (Games Played)", fontsize=13)
plt.show()

There seem to be some outliers where players did not play the majority of the season, yet were given large salaries. This is likely due to season ending injuries. Additionally, there are players present in the data set that were on 10-day contracts. For this reason, we will remove data from players who played in less than 20 games.

In [ ]:
# drop less than 20 games
nba_df = nba_df[nba_df['GP'] >= 20]

### Normalize

In [ ]:
sns.distplot(nba_df["Salary"], fit=norm)

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(nba_df["Salary"])
print("\n mu = {:.2f} and sigma = {:.2f}\n".format(mu, sigma))

# Now plot the distribution
plt.legend(
    ["Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )".format(mu, sigma)], loc="best"
)
plt.ylabel("Frequency")
plt.title("Salary distribution")

# Get also the QQ-plot
fig = plt.figure()
res = probplot(nba_df["Salary"], plot=plt)
plt.show()

In [ ]:
# We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
nba_df["Salary_normalized"] = np.log1p(nba_df["Salary"])

# Check the new distribution
sns.distplot(nba_df["Salary_normalized"], fit=norm)

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(nba_df["Salary_normalized"])
print("\n mu = {:.2f} and sigma = {:.2f}\n".format(mu, sigma))

# Now plot the distribution
plt.legend(
    ["Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )".format(mu, sigma)], loc="best"
)
plt.ylabel("Frequency")
plt.title("Salary distribution")

# Get also the QQ-plot
fig = plt.figure()
res = probplot(nba_df["Salary_normalized"], plot=plt)
plt.show()

## Data Analysis and Visualization

In [ ]:
# scatterplot
sns.set()
cols = [
    "Salary_normalized",
    "Age",
    "MP",
    "3P",
    "TRB",
    "AST",
    "PTS",
    "PER",
    "TS%",
    "DWS",
    "VORP"
]
sns.pairplot(nba_df[cols], size=2.5)
plt.show();

In [ ]:
# Exclude non-numeric columns
numeric_df = nba_df.select_dtypes(include=[np.number])
corrmat = numeric_df.corr()

f, ax = plt.subplots(figsize=(15, 12))
sns.heatmap(corrmat, vmax=0.8, square=True);

In [ ]:
salary_correlations = corrmat['Salary']
print(salary_correlations.sort_values(ascending=False))

In [ ]:
# Visualize number of players at each position by age

plt.figure(figsize=(20,4))
sns.set_style('whitegrid')
sns.countplot(x='Age',hue='Position', data=nba_df, palette='viridis');

#### Target Variable Visualizations

In [ ]:
# boxplot to visualize the spread of salaries by each position
sns.boxplot(x='Position', y='Salary', data=nba_df, palette='rainbow');

In [ ]:
# plot to show correclation between points and salaries by position as well
# points has the highest positive correlation to salary as seen above
sns.lmplot(y='Salary', x='PTS', data=nba_df, hue='Position', palette='Set1');

Now lets compare Salary to VORP.
VORP is a box score estimate of the points per 100 team possessions that a player contributes above a replacement level player, translated to an average team and proportional to an 82 game season.

In [ ]:
sns.jointplot(x='VORP',y='Salary_normalized',data=nba_df,color='purple');

Now lets compare Salary to a defensive advanced statistic like DWS.
DWS stands for Defensive Win Shares, which is a metric in the NBA that compares a player's defensive rating to the league average.

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='DWS', y='Salary_normalized', data=nba_df, hue='Position', palette='viridis', alpha=0.6);

#### Encode categorical varibales

We will not be using the columns, PlayerName, Team, or Position, in our predictive model, as they have little to no correlation to Salary. Thus, we will bypass encoding this data to simplify data management.

#### Feature Selection

#### Step 1: Remove Highly Correlated Features
First, we calculate the correlation matrix to identify pairs of features that are highly correlated (correlation coefficient > 0.8). Highly correlated features can introduce multicollinearity, which negatively impacts model performance. We keep only one feature from each pair of highly correlated features to avoid redundancy.

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

# Find highly correlated features

corr_threshold = 0.8  
correlated_features = set()

for i in range(len(corrmat.columns)):
    for j in range(i):
        if abs(corrmat.iloc[i, j]) > corr_threshold:
            colname_i = corrmat.columns[i]
            colname_j = corrmat.columns[j]
            # Keep one feature and add the other to the set of correlated features to be dropped
            if colname_i not in correlated_features:
                correlated_features.add(colname_j)

# Drop the correlated features
numeric_df_filtered = numeric_df.drop(columns=correlated_features)
numeric_df_filtered.drop(['Salary_normalized'], axis=1, inplace=True)

#### Step 2: Lasso Regression for Additional Feature Selection
Next, we apply Lasso regression, a linear model that includes L1 regularization. Lasso regression not only helps in reducing overfitting but also performs feature selection by shrinking less important feature coefficients to zero. We use LassoCV, which includes cross-validation to find the optimal regularization parameter.

We then use SelectFromModel to identify and retain only the most significant features based on the Lasso regression coefficients.

In [ ]:
# LASSO Regression for additional feature selection
lasso = LassoCV()
lasso.fit(numeric_df_filtered, nba_df['Salary'])

# Use SelectFromModel to get selected features based on LASSO coefficients
sfm = SelectFromModel(lasso, prefit=True)
selected_features_lasso = numeric_df_filtered.columns[sfm.get_support()]

# Convert to a DataFrame if needed
selected_features_df = pd.DataFrame(list(selected_features_lasso), columns=['Selected_Features'])

selected_features_df

#### Significance
By combining correlation-based feature removal with Lasso regression, we ensure that our model is trained on the most relevant features, improving its predictive power and robustness. This process is significant for predicting NBA salaries because it allows the model to focus on the key factors that truly influence player salaries, leading to more accurate and reliable predictions.

In [ ]:
# identify skewness
skewed_feats = (
    numeric_df
    .apply(lambda x: skew(x.dropna()))
    .sort_values(ascending=False)
)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({"Skew": skewed_feats})
skewness.head(20)

In [ ]:
skewness["Skew"].head(10).plot(
    kind="barh", figsize=(20, 10)
).invert_yaxis()  # top 10 skewed columns
plt.xlabel("Skew")
plt.ylabel("Variable Name")
plt.title("Top 10 Skewed Variables")
plt.show()

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print(
    "There are {} skewed numerical features to Box Cox transform (normalize)".format(
        skewness.shape[0]
    )
)

In [ ]:
negative_value_columns = numeric_df.columns[(numeric_df < 0).any()]

# Print the list of column names
print("Columns with negative values:")
print(negative_value_columns.tolist())


In [ ]:
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    # skip over columns that don't need transformation
    # skip over columns that have negative values so that they don't become NULL when transforming
    if feat not in [
        "Id",
        "Salary",
        "Salary_normalized",
        'OWS', 
        'WS', 
        'WS/48', 
        'OBPM', 
        'DBPM', 
        'BPM', 
        'VORP'
    ]:
        nba_df[feat] = boxcox1p(nba_df[feat], lam)

In [ ]:
# check that the box cot did not add any NULL values
null_columns = nba_df.columns[nba_df.isnull().any()]
null_count = nba_df[null_columns].isnull().sum()

print("Column Name: NULL Count")
for i in range(0, len(null_columns)):
    print(f"{null_columns[i]}: {null_count[i]}")

## Data Analytics

All of our data is labled therefore we will be implementing supervised learning methods

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import lightgbm as lgb
from skopt import BayesSearchCV

In [ ]:
def train_and_compute_rmse(df, model):
    # Splitting the data into train and test set  
    X = df.drop(columns=['Salary', 'Id', 'PlayerName', 'Position', 'Team', 'Salary_normalized'])
    y = df['Salary']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Training the model
    model.fit(X_train, np.log1p(y_train))
    
    # Making predictions on the test set
    y_pred_log = model.predict(X_test)
    y_pred = np.expm1(y_pred_log)

    if (
            np.isnan(y_pred).any()
            or np.isinf(y_pred).any()
        ):
            print(
                f"Warning: NaN or infinity values found in predictions or true values. Imputing 0 for problematic values in y_pred."
            )
            y_pred[np.isnan(y_pred) | np.isinf(y_pred)] = 0

    # Computing RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    return rmse

In [ ]:
def compute_rmse_std(df, model):
    rmse_list = []
    for i in range(30):
        rmse_list.append(train_and_compute_rmse(df, model))

    mean = np.mean(rmse_list)
    std = np.std(rmse_list)

    return mean, std

In [ ]:
lr_w_int = LinearRegression()
lr_no_int = LinearRegression(fit_intercept=False)
elastic_net = ElasticNet(alpha=0.01, l1_ratio=0.1)

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=10)

In [ ]:
rf = RandomForestRegressor(n_estimators=500)

In [ ]:
dt = DecisionTreeRegressor(max_depth=10)

In [ ]:
model_xgb = xgb.XGBRegressor(max_depth=5, n_estimators=1000, learning_rate=0.01)

In [ ]:
model_lgb = lgb.LGBMRegressor(max_depth=5, n_estimators=1000, learning_rate=0.01)

In [ ]:
#Get the avg rmse and std over 30 tests for each model
lr_no_int_list = compute_rmse_std(nba_df, lr_no_int)
lr_w_int_list = compute_rmse_std(nba_df, lr_w_int)
elastic_net_list = compute_rmse_std(nba_df, elastic_net)
#neigh_list = compute_rmse_std(nba_df, neigh)
dt_list = compute_rmse_std(nba_df, dt)
rf_list = compute_rmse_std(nba_df, rf)
model_xgb_list = compute_rmse_std(nba_df, model_xgb)

# plot RMSE and STD for each Algorithm
data = {
    "Linear (No Intercept)": lr_no_int_list,
    "Linear (w/ Intercept)": lr_w_int_list,
    "Elastic Net": elastic_net_list,
    #"Nearest Neighbor": neigh_list,
    "Decision Tree": dt_list,
    "Random Forest": rf_list,
    "XGBoost": model_xgb_list,
}
data_df = pd.DataFrame(data=data).T.reset_index().sort_values(by=[0], ascending=True)
data_df.columns = ["Algorithm", "RMSE", "STD"]

In [ ]:
# creating the bar plot
data_df.plot(kind="bar", x="Algorithm", y=["RMSE", "STD"], figsize=(20, 10), rot=0)
plt.xlabel("Algorithm", fontsize=20)
plt.ylabel("Root Mean Squared Error / Standard Deviation", fontsize=20)
plt.show()

In [ ]:
def hyperparameter_tune_bayesian(X_train, y_train, regressor):
    """
    Perform hyperparameter tuning for XGBoost or LightGBM using Bayesian search.

    Parameters:
    - X_train: pandas DataFrame
        Training features.
    - y_train: pandas Series
        Training target variable.
    - regressor_type: str
        Type of regressor to tune ('xgboost' or 'lightgbm').

    Returns:
    - best_params: dict
        Best hyperparameters found during tuning.
    """
    # Define the common parameter space for both XGBoost and LightGBM
    param_space_common = {
        "n_estimators": (100, 1200),
        "learning_rate": (0.01, 0.2, "log-uniform"),
        "max_depth": (3, 10),
    }

    regressor_type = regressor.lower()
    if regressor_type == "xgboost":
        regressor = xgb.XGBRegressor()
    elif regressor_type == "lightgbm":
        regressor = lgb.LGBMRegressor()
    else:
        raise ValueError("Unsupported regressor type. Choose 'xgboost' or 'lightgbm'.")

    # Update the search space with common parameters
    param_space = param_space_common.copy()

    # Perform Bayesian search
    bayes_search = BayesSearchCV(
        estimator=regressor,
        search_spaces=param_space,
        scoring="neg_mean_squared_error",
        cv=5,
        n_jobs=-1,  # Set the number of parallel jobs
    )
    bayes_search.fit(X_train, np.log1p(y_train))

    # Get the best hyperparameters
    best_params = bayes_search.best_params_

    return best_params

In [ ]:
def k_fold_regression(
    data,
    regressor,
    target_column="Salary",
    cols_to_ignore=['Salary', 'Id', 'PlayerName', 'Position', 'Team', 'Salary_normalized'],
    n_splits=5,
    tune_hyperparameters=False,
):
    rmse_scores = []
    train_sizes = []
    test_sizes = []

    X = data.drop(columns=cols_to_ignore)
    y = data[target_column]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    kf = KFold(n_splits=5, shuffle=True) 

    for train_index, val_index in kf.split(X_train):
        if (
            isinstance(regressor, (xgb.XGBRegressor, lgb.LGBMRegressor))
            and tune_hyperparameters
        ):  # Add LGBMRegressor to the isinstance check
            # Determine the regressor_type based on the type of the regressor
            if isinstance(regressor, xgb.XGBRegressor):
                regressor_type = "XGBoost"
            elif isinstance(regressor, lgb.LGBMRegressor):
                regressor_type = "LightGBM"
            else:
                raise ValueError(
                    "Unsupported regressor type. Supported types: XGBRegressor, LGBMRegressor"
                )

            X_train_hyper, y_train_hyper = (
                data.drop(cols_to_ignore, axis=1),
                data[target_column],
            )
            best_params = hyperparameter_tune_bayesian(
                X_train_hyper, y_train_hyper, regressor_type
            )  # Specify 'xgboost' or 'lightgm' as the regressor type
            print(
                f"Best hyperparameters for {regressor_type} Fold: {best_params}"
            )
            regressor.set_params(**best_params)  # Set the best hyperparameters


        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        regressor.fit(X_train_fold, y_train_fold)
        y_pred_fold = regressor.predict(X_val_fold)
        rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_fold))
        rmse_scores.append(rmse)
        train_sizes.append(len(y_train_fold))
        test_sizes.append(len(y_val_fold))


    # Print RMSE scores for each fold
    # for fold, rmse in enumerate(rmse_scores):
    #     print(f"Fold {fold+1} RMSE: {rmse}")

    return rmse_scores, train_sizes, test_sizes


In [ ]:
def compute_rmse_std_k_fold(df, model, tune_hyper=False):
    rmse_list = []
        
    rmse_list.append(k_fold_regression(df, model, tune_hyperparameters=tune_hyper))

    mean = np.mean(rmse_list)
    std = np.std(rmse_list)

    return mean, std

In [ ]:
def print_rmse_k_fold(data, regressor, target_column="Salary", cols_to_ignore=['Salary', 'Id', 'PlayerName', 'Position', 'Team', 'Salary_normalized'], n_splits=5, tune_hyperparameters=False):
    rmse_scores, train_sizes, test_sizes = k_fold_regression(
        data, regressor, target_column, cols_to_ignore, n_splits, tune_hyperparameters
    )

    for fold, (rmse, train_size, test_size) in enumerate(zip(rmse_scores, train_sizes, test_sizes)):
        print(f"Fold {fold + 1} RMSE: {rmse:.4f}, Train Size: {train_size}, Test Size: {test_size}")

    overall_rmse = np.mean(rmse_scores)
    print(f"Overall RMSE: {overall_rmse:.4f}")

In [ ]:
#Get the avg rmse and std over 30 tests for each model
lr_no_int_list = compute_rmse_std_k_fold(nba_df, lr_no_int)
lr_w_int_list = compute_rmse_std_k_fold(nba_df, lr_w_int)
elastic_net_list = compute_rmse_std_k_fold(nba_df, elastic_net)
#neigh_list = compute_rmse_std_k_fold(nba_df, neigh)
dt_list = compute_rmse_std_k_fold(nba_df, dt)
rf_list = compute_rmse_std_k_fold(nba_df, rf)
model_xgb_list = compute_rmse_std_k_fold(nba_df, model_xgb)

#The next line takes a while (Roughly 10 mins), If want to quickly run, comment out this line and the line below in data
#model_xgb_hyper_list = compute_rmse_std_k_fold(nba_df, model_xgb, tune_hyper=True)

# plot RMSE and STD for each Algorithm
data = {
    "Linear (No Intercept)": lr_no_int_list,
    "Linear (w/ Intercept)": lr_w_int_list,
    "Elastic Net": elastic_net_list,
    #"Nearest Neighbor": neigh_list,
    "Decision Tree": dt_list,
    "Random Forest": rf_list,
    "XGBoost": model_xgb_list,

    #Comment below if want to run quicker
    #"XGBoost Hyper": model_xgb_hyper_list,
}
data_df = pd.DataFrame(data=data).T.reset_index().sort_values(by=[0], ascending=True)
data_df.columns = ["Algorithm", "RMSE", "STD"]

In [ ]:
# creating the bar plot
data_df.plot(kind="bar", x="Algorithm", y=["RMSE", "STD"], figsize=(20, 10), rot=0)
plt.xlabel("Algorithm", fontsize=20)
plt.ylabel("Root Mean Squared Error / Standard Deviation", fontsize=20)
plt.show()

### Meta Model

In [ ]:
from sklearn.ensemble import StackingRegressor

# first stacking model
  
estimators = [
   ('elastic_net', elastic_net),
   ('model_xgb', model_xgb),
   ('lr_w_int', lr_w_int)
]


sr = StackingRegressor(
   estimators=estimators,
   final_estimator=rf
)

In [ ]:
from sklearn.ensemble import VotingRegressor

# voting stacking model, putting weights on different models

vr = VotingRegressor([
   ('rf', rf),
   ('model_xgb', model_xgb),
   ('elastic_net', elastic_net),
   ('lr_w_int', lr_w_int)
  
], weights=[1,2,2,1])


In [ ]:
estimators2 = [
   ('elastic_net', elastic_net),
   ('model_xgb', model_xgb),
   ('lr_w_int', lr_w_int),
   ('rf', rf)
]

# using the voting model as our final estimator

sr2 = StackingRegressor(
   estimators=estimators2,
   final_estimator=vr
)

# More tesing with the new models
Removed Nearest Neighbor, Linear No Int, Linear W Int, Elastic Net, and Descision tree because they are worst performing

In [ ]:
#Get the avg rmse and std over 30 tests for each model
rf_list = compute_rmse_std_k_fold(nba_df, rf)
model_xgb_list = compute_rmse_std_k_fold(nba_df, model_xgb)
sr_list = compute_rmse_std_k_fold(nba_df, sr)
vr_list = compute_rmse_std_k_fold(nba_df, vr)
sr2_list = compute_rmse_std_k_fold(nba_df, sr2)
model_xgb_hyper_list = compute_rmse_std_k_fold(nba_df, model_xgb, tune_hyper=True)
sr_hyper_list = compute_rmse_std_k_fold(nba_df, sr, tune_hyper=True)
vr_hyper_list = compute_rmse_std_k_fold(nba_df, vr, tune_hyper=True)
sr2_hyper_list = compute_rmse_std_k_fold(nba_df, sr2, tune_hyper=True)


# plot RMSE and STD for each Algorithm
data = {
    "Random Forest": rf_list,
    "XGBoost": model_xgb_list,
    "Stacking Regressor": sr_list,
    "Voting Regressor": vr_list,
    "Stacking Regressor 2": sr2_list,
    "XGBoost Hyper": model_xgb_hyper_list,
    "Stacking Regressor Hyper": sr_hyper_list,
    "Voting Regressor Hyper": vr_hyper_list,
    "Stacking Regressor 2 Hyper": sr2_hyper_list,
}
data_df = pd.DataFrame(data=data).T.reset_index().sort_values(by=[0], ascending=True)
data_df.columns = ["Algorithm", "RMSE", "STD"]

In [ ]:
# creating the bar plot
data_df.plot(kind="bar", x="Algorithm", y=["RMSE", "STD"], figsize=(20, 10), rot=0)
plt.xlabel("Algorithm", fontsize=20)
plt.ylabel("Root Mean Squared Error / Standard Deviation", fontsize=20)
plt.show()